In [2]:
import asyncio, aiohttp, nest_asyncio, pandas as pd, json, os
from understat import Understat
nest_asyncio.apply()

os.makedirs("data/raw", exist_ok=True)

async def get_shots(mid:int):
    async with aiohttp.ClientSession() as s:
        u = Understat(s)
        return await u.get_match_shots(mid)

def flatten_shots(shots):
    rows=[]
    for side in ('h','a'):
        for d in shots[side]:
            rows.append({
                'side': side,
                'team': d.get('team'),
                'player': d.get('player'),
                'minute': int(d.get('minute', 0)),
                'xG': float(d.get('xG', 0)),
                'result': d.get('result'),
                'situation': d.get('situation'),
                'shotType': d.get('shotType'),
                'x': float(d.get('X', 0)),
                'y': float(d.get('Y', 0)),
            })
    return pd.DataFrame(rows)

def pull_and_save(mid:int):
    shots = asyncio.get_event_loop().run_until_complete(get_shots(mid))
    df = flatten_shots(shots).sort_values('xG', ascending=False).reset_index(drop=True)
    # save both raw and tidy
    with open(f"data/raw/understat_shots_match_{mid}.json","w") as f:
        json.dump(shots, f)
    df.to_csv(f"data/raw/understat_shots_match_{mid}.csv", index=False)
    return df

# Add all match IDs below manually as the season goes.
match_ids = [28778 ]  # add more: [26975, 27001, 27002, ...]

# run for each
tables = {}
for mid in match_ids:
    tables[mid] = pull_and_save(mid)

# show the first table you pulled (sorted by xG)
next(iter(tables.values())).head(10)

,side,team,player,minute,xG,result,situation,shotType,x,y
0,h,None,Hugo Ekitike,38,0.534849,MissedShots,OpenPlay,Head,0.942,0.483
1,a,None,Antoine Semenyo,63,0.425994,Goal,OpenPlay,LeftFoot,0.908,0.513
2,h,None,Hugo Ekitike,36,0.404091,Goal,OpenPlay,RightFoot,0.891,0.524
3,a,None,Marcus Tavernier,34,0.351001,SavedShot,OpenPlay,RightFoot,0.870,0.521
4,a,None,Antoine Semenyo,5,0.316921,MissedShots,OpenPlay,RightFoot,0.908,0.444
5,h,None,Virgil van Dijk,4,0.257975,MissedShots,FromCorner,Head,0.914,0.505
6,h,None,Mohamed Salah,93,0.138905,Goal,OpenPlay,RightFoot,0.899,0.613
7,h,None,Hugo Ekitike,46,0.134705,MissedShots,FromCorner,OtherBodyPart,0.961,0.521
8,h,None,Mohamed Salah,47,0.113376,BlockedShot,OpenPlay,LeftFoot,0.850,0.427
9,h,None,Florian Wirtz,62,0.111039,MissedShots,OpenPlay,RightFoot,0.840,0.458
